In [1]:
import csv
import json
import pymysql
import pandas as pd
import requests
from time import gmtime, strftime
import dtale
#connecting to goDaddy
import pymysql
import pandas as pd
import yfinance as yf
import pymysql
import pandas as pd

conn=pymysql.connect(host='148.66.134.234',port=int(3306),user='unhotel_user',db='unhotel_nodal_data_trading',passwd='Nodal@123456')
cur = conn.cursor()

#defining symnols 
symbol=['ADANIPORTS', 'ASIANPAINT', 'AUBANK', 'AXISBANK', 'BAJAJ-AUTO', 'BAJAJFINSV', 'BAJFINANCE', 'BANDHANBNK', 'BHARTIARTL',
       'BPCL', 'BRITANNIA', 'CIPLA', 'COALINDIA', 'DIVISLAB', 'DRREDDY', 'EICHERMOT', 'FEDERALBNK', 'GRASIM', 'HCLTECH', 'HDFC',
       'HDFCBANK', 'HDFCLIFE', 'HEROMOTOCO', 'HINDALCO', 'HINDUNILVR', 'ICICIBANK', 'IDFCFIRSTB', 'INDUSINDBK', 'INFY', 'IOC', 
       'ITC', 'JSWSTEEL', 'KOTAKBANK', 'LT', 'MARUTI', 'NESTLEIND', 'NTPC', 'ONGC', 'PNB', 'POWERGRID', 'RBLBANK', 'RELIANCE', 
        'SBILIFE', 'SBIN', 'SHREECEM', 'SUNPHARMA','TATACONSUM', 'TATAMOTORS', 'TATASTEEL', 'TCS', 'TECHM', 'TITAN', 'ULTRACEMCO', 'UPL', 'WIPRO']

#storing data

try:
    df=pd.DataFrame()
    for j in range(0,len(symbol)):
        url = "https://nse-data1.p.rapidapi.com/equity_option_chain"
    
        querystring = {"symbol":symbol[j]}
        
        headers = {
            'x-rapidapi-host': "nse-data1.p.rapidapi.com",
            'x-rapidapi-key': "c84f011115mshff0f17677ab87b0p14d794jsn12853448e7cb"
            }
        df=pd.DataFrame()
        response = requests.request("GET", url, headers=headers, params=querystring)
        json_dict = json.loads(response.text)
        for i in range(len(json_dict['body']['records']['data'])):
        
            df1=pd.json_normalize(json_dict['body']['records']['data'][i])
            df = pd.concat([df, df1], axis=0,ignore_index=False)
        
        df['timestamp'] = [strftime("%Y-%m-%d %H:%M:%S", gmtime())] * df.shape[0]
#         pd.set_option("max_columns",None)
#         pd.set_option("max_rows",None)
        strike_price_gap=int(df.strikePrice.unique()[1]-df.strikePrice.unique()[0])
        no_of_expiry=df['expiryDate'].nunique()
        distinct_strikeprice=df.strikePrice.unique()
        
        df=df[df['strikePrice']>=round(df['CE.underlyingValue'].mean(),-1)-strike_price_gap*10]
        df=df[df['strikePrice']<=round(df['CE.underlyingValue'].mean(),-1)+strike_price_gap*10]
        df.rename(columns = {'strikePrice':'strikeprice', 'expiryDate' : 'expirydate', 
                                        'CE.strikePrice' : 'ce_strikeprice','CE.expiryDate' : 'ce_expirydate','CE.underlying' : 'ce_underlying', 
                                        'CE.identifier' : 'ce_identifier',  'CE.askPrice':'ce_askprice','CE.underlyingValue':'ce_underlyingvalue',
                                        'PE.strikePrice' : 'pe_strikeprice','PE.expiryDate':'pe_expirydate',
                                        'PE.underlying':'pe_underlying','PE.identifier':'pe_identifier','PE.totalSellQuantity':'pe_totalsellquantity',
                                        'PE.askQty':'pe_askqty','PE.askPrice':'pe_askprice','PE.underlyingValue':'pe_underlyingvalue',
                                 'CE.totalSellQuantity':'ce_totalsellquantity','CE.bidQty':'ce_bidqty','CE.bidprice':'ce_bidprice','CE.askQty':'ce_askqty',
                                 'CE.totalBuyQuantity':'ce_totalbuyquantity'}, inplace = True)
          
        cols=['timestamp','strikeprice', 'expirydate', 'ce_strikeprice', 'ce_expirydate','ce_underlying', 'ce_identifier','ce_totalsellquantity','ce_askqty',
                   'ce_askprice','ce_underlyingvalue','pe_strikeprice','pe_expirydate','pe_identifier',
                   'pe_totalsellquantity','pe_askqty','pe_askprice', 'pe_underlyingvalue', 'ce_bidqty',
             'ce_bidprice', 'ce_totalbuyquantity']  
        # df.where(filter1 & filter2, inplace = True)
        df=df[cols].dropna()
        for i,row in df.iterrows():
            
            sql = """INSERT INTO rapid_api_option_chain(timestamp, strikeprice,expirydate, ce_strikeprice, ce_expirydate, ce_underlying, ce_identifier, ce_totalsellquantity, ce_askqty,ce_askprice,
      ce_underlyingvalue  ,pe_strikeprice,pe_expirydate ,pe_identifier,pe_totalsellquantity,pe_askqty,pe_askprice,pe_underlyingvalue,ce_bidqty,ce_bidprice,ce_totalbuyquantity) 
                 VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
            cur.execute(sql,tuple(row))
        #print("Record Added", i)
        conn.commit()


except:
    df=pd.read_sql_query("SELECT * FROM rapid_api_option_chain ORDER BY ID DESC LIMIT 1",conn)
    a=df['ce_underlying'][0]
    if(symbol.index(a) != 56):
        for k in range(symbol.index(a),52):
            url = "https://nse-data1.p.rapidapi.com/equity_option_chain"
        
            querystring = {"symbol":symbol[k]}
            
            headers = {
                'x-rapidapi-host': "nse-data1.p.rapidapi.com",
                'x-rapidapi-key': "c84f011115mshff0f17677ab87b0p14d794jsn12853448e7cb"
                }
            df=pd.DataFrame()
            response = requests.request("GET", url, headers=headers, params=querystring)
            json_dict = json.loads(response.text)
            for i in range(len(json_dict['body']['records']['data'])):
            
                df1=pd.json_normalize(json_dict['body']['records']['data'][i])
                df = pd.concat([df, df1], axis=0,ignore_index=False)
            
            df['timestamp'] = [strftime("%Y-%m-%d %H:%M:%S", gmtime())] * df.shape[0]
            pd.set_option("max_columns",None)
            pd.set_option("maxmrows",None)
            strike_price_gap=int(df.strikePrice.unique()[1]-df.strikePrice.unique()[0])
            no_of_expiry=df['expiryDate'].nunique()
            distinct_strikeprice=df.strikePrice.unique()
            
            df=df[df['strikePrice']>=round(df['CE.underlyingValue'].mean(),-1)-strike_price_gap*10]
            df=df[df['strikePrice']<=round(df['CE.underlyingValue'].mean(),-1)+strike_price_gap*10]
            df.rename(columns = {'strikePrice':'strikeprice', 'expiryDate' : 'expirydate', 
                                            'CE.strikePrice' : 'ce_strikeprice','CE.expiryDate' : 'ce_expirydate','CE.underlying' : 'ce_underlying', 
                                            'CE.identifier' : 'ce_identifier',  'CE.askPrice':'ce_askprice','CE.underlyingValue':'ce_underlyingvalue',
                                            'PE.strikePrice' : 'pe_strikeprice','PE.expiryDate':'pe_expirydate',
                                            'PE.underlying':'pe_underlying','PE.identifier':'pe_identifier','PE.totalSellQuantity':'pe_totalsellquantity',
                                            'PE.askQty':'pe_askqty','PE.askPrice':'pe_askprice','PE.underlyingValue':'pe_underlyingvalue',
                                     'CE.totalSellQuantity':'ce_totalsellquantity','CE.bidQty':'ce_bidqty','CE.bidprice':'ce_bidprice','CE.askQty':'ce_askqty',
                                     'CE.totalBuyQuantity':'ce_totalbuyquantity'}, inplace = True)
              
            cols=['timestamp','strikeprice', 'expirydate', 'ce_strikeprice', 'ce_expirydate','ce_underlying', 'ce_identifier','ce_totalsellquantity','ce_askqty',
                       'ce_askprice','ce_underlyingvalue','pe_strikeprice','pe_expirydate','pe_identifier',
                       'pe_totalsellquantity','pe_askqty','pe_askprice', 'pe_underlyingvalue', 'ce_bidqty',
                 'ce_bidprice', 'ce_totalbuyquantity']  
            # df.where(filter1 & filter2, inplace = True)
            df=df[cols].dropna()
            for i,row in df.iterrows():
                
                sql = """INSERT INTO rapid_api_option_chain(timestamp, strikeprice,expirydate, ce_strikeprice, ce_expirydate, ce_underlying, ce_identifier, ce_totalsellquantity, ce_askqty,ce_askprice,
          ce_underlyingvalue  ,pe_strikeprice,pe_expirydate ,pe_identifier,pe_totalsellquantity,pe_askqty,pe_askprice,pe_underlyingvalue,ce_bidqty,ce_bidprice,ce_totalbuyquantity) 
                     VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
                cur.execute(sql,tuple(row))
            #print("Record Added", i)
            conn.commit()        
        
        